In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import resample

Calcolare vittorie su singola superficie

In [ ]:
def calculate_surface_wins(df, players_df):
    surface_wins = df.groupby(['Winner', 'Surface']).size().unstack(fill_value=0)
    surface_wins.columns = [f"{surface}_wins" for surface in surface_wins.columns]
    return players_df.merge(surface_wins, how='left', left_on='Player_name', right_index=True).fillna(0)


Calcolare vittorie indoor e outdoor

In [ ]:
def calculate_court_wins(df, players_df):
    court_wins = df.groupby(['Winner', 'Court']).size().unstack(fill_value=0)
    court_wins.columns = [f"{court}_wins" for court in court_wins.columns]
    return players_df.merge(court_wins, how='left', left_on='Player_name', right_index=True).fillna(0)


In [ ]:
def add_surface_ratios(players_df):
    """
    Aggiunge le feature di ratio per ogni superficie al DataFrame.
    Ogni ratio è il rapporto tra i match vinti e i match giocati per la superficie,
    con gestione dei casi in cui il denominatore è 0.
    """
    # Evita divisioni per 0 con un controllo diretto
    players_df['Clay_ratio'] = players_df['Clay_wins'] / players_df['Clay_matches'].replace(0, 1)
    players_df['Hard_ratio'] = players_df['Hard_wins'] / players_df['Hard_matches'].replace(0, 1)
    players_df['Grass_ratio'] = players_df['Grass_wins'] / players_df['Grass_matches'].replace(0, 1)

    # Gestione NaN o inf residui
    players_df.fillna(0, inplace=True)

    return players_df

#TODO: aggiungi soglia di almeno tot. partite, sennò butta la riga del df

Punti totali accumulati su ogni superficie

In [ ]:
def calculate_surface_points(df, players_df):
    # Definizione dei punti in base al tipo di torneo e al round
    points_table = {
        "Grand Slam": {
            "Winner": 2000,
            "The Final": 1200,
            "Semifinals": 720,
            "Quarterfinals": 360,
            "2nd Round": 180,
            "1st Round": 10,
        },
        "Masters 1000": {
            "Winner": 1000,
            "The Final": 600,
            "Semifinals": 360,
            "Quarterfinals": 180,
            "2nd Round": 90,
            "1st Round": 10,
        },
        "ATP500": {
            "Winner": 500,
            "The Final": 300,
            "Semifinals": 180,
            "Quarterfinals": 90,
            "2nd Round": 45,
            "1st Round": 10,
        },
        "ATP250": {
            "Winner": 250,
            "The Final": 150,
            "Semifinals": 90,
            "Quarterfinals": 45,
            "2nd Round": 20,
            "1st Round": 5,
        },
    }

    # Calcola i punti per ogni match basandosi sul tipo di torneo e sul round
    def get_points(row):
        tournament_type = row['Series']
        round_type = row['Round']
        return points_table.get(tournament_type, {}).get(round_type, 0)

    # Applica la funzione per calcolare i punti per ogni riga
    df['Round_points'] = df.apply(get_points, axis=1)

    # Calcola i punti totali guadagnati su ciascuna superficie
    surface_points = df.groupby(['Winner', 'Surface'])['Round_points'].sum().unstack(fill_value=0)
    surface_points.columns = [f"{surface}_points" for surface in surface_points.columns]

    # Merge con il DataFrame dei giocatori
    players_df = players_df.merge(surface_points, how='left', left_on='Player_name', right_index=True).fillna(0)

    return players_df


Numero di Match Giocati per Superficie

In [ ]:
def calculate_surface_matches(df, players_df):
    surface_matches = df.groupby(['P1_name', 'Surface']).size().unstack(fill_value=0)
    surface_matches.columns = [f"{surface}_matches" for surface in surface_matches.columns]
    return players_df.merge(surface_matches, how='left', left_on='Player_name', right_index=True).fillna(0)


Anni di Attività

In [ ]:
def calculate_years_active(df, players_df):
    years_active = df.groupby('P1_name')['Year'].nunique().rename('Years_active')
    return players_df.merge(years_active, how='left', left_on='Player_name', right_index=True).fillna(0)



Calcolo superficie preferita

In [ ]:
def calculate_preferred_surface(players_df):
    # Somma normalizzata per combinare vittorie e punti
    players_df['Clay_score'] = players_df['Clay_wins'] + players_df['Clay_points'] / 250
    players_df['Hard_score'] = players_df['Hard_wins'] + players_df['Hard_points'] / 250
    players_df['Grass_score'] = players_df['Grass_wins'] + players_df['Grass_points'] / 250

    # Determina la superficie preferita
    players_df['Preferred_surface'] = players_df[['Clay_score', 'Hard_score', 'Grass_score']].idxmax(axis=1)
    players_df['Preferred_surface'] = players_df['Preferred_surface'].str.replace('_score', '')

    # Rimuove colonne temporanee
    players_df.drop(columns=['Clay_score', 'Hard_score', 'Grass_score'], inplace=True)
    #TODO: Spiegare bene cosa intendi per preferita
    return players_df

Genero il df finale

In [ ]:
def generate_features_and_target(df, players_df):
    # Calcola le feature
    players_df = calculate_surface_wins(df, players_df)
    players_df = calculate_court_wins(df, players_df)
    players_df = calculate_surface_points(df, players_df)
    players_df = calculate_surface_matches(df, players_df)
    players_df = calculate_years_active(df, players_df)
    
    # Determina la superficie preferita
    players_df = calculate_preferred_surface(players_df)
    
    print(f"[DEBUG] Dataframe finale con feature: {players_df.shape[0]} righe, {players_df.shape[1]} colonne")
    print(players_df.head(50))
    return players_df


# Carico dataset

In [ ]:
# Carica il dataset e crea il dataframe che utilizzo per l'addestramento
print("Caricamento del dataset")
df = pd.read_csv('./atp_tennis.csv')

# Check dataset

In [ ]:
# Esplora le prime righe del dataset
print("Prime righe del dataset:")
print(df.head())

# Mostra informazioni sulle colonne
print("\nInformazioni sul dataset:")
print(df.info())

# Conta i valori nulli in ciascuna colonna
print("\nValori nulli per colonna:")
print(df.isnull().sum())


# Pulizia valori vuoti e filtraggio, aggiunta features

In [ ]:
# Converti la colonna Date in formato datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
# Aggiungi la colonna Year estraendola dalla colonna Date
df['Year'] = df['Date'].dt.year
# Filtra solo gli ultimi 5 anni
latest_year = df['Year'].max()

In [ ]:
# Mostra le righe totali prima del filtraggio
print("\nPrima del filtraggio e pulizia:")
print(f"Righe totali: {len(df)}")

# Filtra le quote negative o nulle
df = df[(df['Odd_1'] > 0) & (df['Odd_2'] > 0)]
# Filtra i punti ATP positivi
df = df[(df['Pts_1'] > 0) & (df['Pts_2'] > 0)]

# Rimuovi righe con valori NaN in qualsiasi colonna
df = df.dropna()


print(f"Dati filtrati per gli ultimi 5 anni: {df_recent.shape}")


# Rinominare le colonne
df_recent.rename(
    columns={
        'Pts_1': 'ATP_pts_p1',
        'Pts_2': 'ATP_pts_p2',
        'Rank_1': 'ATP_rank_p1',
        'Rank_2': 'ATP_rank_p2',
        'Odd_1': 'PreMatch_Odd_p1',
        'Odd_2': 'PreMatch_Odd_p2',
        'Player_1': 'P1_name',
        'Player_2': 'P2_name',
        'Best of': 'N_sets'
    },
    inplace=True
)

# Calcola la differenza di ranking per ogni match
df_recent['Rank_diff'] = abs(df_recent['ATP_rank_p1'] - df_recent['ATP_rank_p2'])

# Stampa le righe totali dopo il filtraggio
print("\nDopo il filtraggio:")
print(f"Righe rimanenti: {len(df_recent)}")

# Stampa il numero di valori mancanti per ogni colonna (dopo il filtraggio, dovrebbe essere 0)
print("\nValori nulli per colonna (dopo il filtraggio):")
print(df_recent.isnull().sum())

# Visualizza le prime righe del DataFrame pulito
print("\nPrime righe del DataFrame pulito:")
print(df_recent.head())



Creo dataframe dei giocatori

In [ ]:
# Estrai tutti i giocatori da P1_name e P2_name
all_players = pd.concat([df_recent['P1_name'], df_recent['P2_name']])

# Rimuovi i duplicati e crea un set unico di giocatori
unique_players = sorted(all_players.unique())

# Crea un nuovo dataframe dei giocatori
players_df = pd.DataFrame(unique_players, columns=['Player_name'])

# Verifica il dataframe creato
print(f"Dataframe dei giocatori creato con {players_df.shape[0]} giocatori unici:")
print(players_df.head(10))


In [ ]:
players_df_final = generate_features_and_target(df_recent, players_df)

# Filtra i giocatori con almeno 30 partite giocate tra Clay, Hard o Grass
players_df_filtered = players_df_final[
    (players_df_final['Clay_matches'] + players_df_final['Hard_matches'] + players_df_final['Grass_matches']) >= 10
]

# Visualizza il numero di giocatori filtrati e le prime righe del nuovo DataFrame
print(f"\n[DEBUG] Numero di giocatori filtrati: {len(players_df_filtered)}")
print(players_df_filtered.head())


In [ ]:
# Applica la funzione al DataFrame
players_df_filtered = add_surface_ratios(players_df_filtered)

# Controlla il risultato
print("\n[DEBUG] Prime righe del DataFrame con i ratio aggiunti:")
print(players_df_filtered[['Player_name', 'Clay_ratio', 'Hard_ratio', 'Grass_ratio']].head(10))

In [ ]:
# Mappatura della colonna Preferred_surface
surface_mapping = {'Clay': 0, 'Hard': 1, 'Grass': 2}
players_df_filtered['Preferred_surface_mapped'] = players_df_filtered['Preferred_surface'].map(surface_mapping)

print("\n[DEBUG] Colonna di target mappata:")
print(players_df_filtered[['Preferred_surface', 'Preferred_surface_mapped']].head())


In [ ]:
from sklearn.model_selection import train_test_split

# Feature selezionate per ridurre la ridondanza
selected_features = [
    'Clay_ratio', 'Hard_ratio', 'Grass_ratio', 'Clay_wins', 'Grass_wins', 'Hard_wins',
]

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.model_selection import train_test_split

# Creazione del DataFrame X e del target
X = players_df_filtered[selected_features]
y = players_df_filtered['Preferred_surface_mapped']

# Suddivisione train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"[DEBUG] Dimensione train set: {X_train.shape}, Dimensione test set: {X_test.shape}")

# Verifica del bilanciamento iniziale
print("Distribuzione dei target nel train set prima del bilanciamento:", Counter(y_train))
print("Distribuzione dei target nel test set:", Counter(y_test))
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE

# Bilanciamento del test set con oversampling semplice
def balance_test_set(X_test, y_test, desired_samples=100):
    test_data = X_test.copy()
    test_data['target'] = y_test
    balanced_test_data = []

    for target in test_data['target'].unique():
        target_data = test_data[test_data['target'] == target]
        upsampled_data = resample(
            target_data,
            replace=True,  # Oversampling con duplicazione
            n_samples=desired_samples,
            random_state=42
        )
        balanced_test_data.append(upsampled_data)

    final_test_data = pd.concat(balanced_test_data)
    X_test_balanced = final_test_data.drop('target', axis=1)
    y_test_balanced = final_test_data['target']
    return X_test_balanced, y_test_balanced

# Applica il bilanciamento al test set
X_test_balanced, y_test_balanced = balance_test_set(X_test, y_test, desired_samples=50)
print("Distribuzione del test set dopo il bilanciamento:", Counter(y_test_balanced))


In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter

# Identifica la classe minoritaria
class_counts = Counter(y_train)
minority_class = min(class_counts, key=class_counts.get)
print(f"Classe minoritaria identificata: {minority_class}")

# Identifica la classe minoritaria e il numero massimo desiderato
desired_samples_per_class = 1000  # Numero di campioni desiderati per ogni classe
desired_samples_per_class_test = 250

# Configura SMOTE per aumentare a 1000 campioni per ogni classe
smote = SMOTE(sampling_strategy={label: desired_samples_per_class for label in Counter(y_train).keys()},
              random_state=42)

# Applica SMOTE al training set
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)


smote_test = SMOTE(sampling_strategy={label: desired_samples_per_class_test for label in Counter(y_test_balanced).keys()},
              random_state=42)
X_test_smote, y_test_smote = smote_test.fit_resample(X_test_balanced, y_test_balanced)

# Controllo del bilanciamento nel test set
print("Distribuzione dei target nel test set (non modificato):", Counter(y_test))
# Verifica della distribuzione dopo il bilanciamento
print("Distribuzione dei target nel test set dopo SMOTE:", Counter(y_test_smote))


# Verifica del bilanciamento dopo SMOTE
print("Distribuzione dei target nel train set dopo SMOTE:", Counter(y_train_smote))


# OPTIONAL: Codice per normalizzare/scalare i dati se necessario
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_smote)
X_test_scaled = scaler.transform(X_test_smote)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Calcola la matrice di correlazione
correlation_matrix = X.corr()

# Visualizza la matrice di correlazione
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)
plt.title("Matrice di Correlazione delle Feature")
plt.show()


In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

# Dizionario dei modelli da testare
models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "SVM": SVC(probability=True, random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42),
    "KNN": KNeighborsClassifier()
}

# Dizionario per memorizzare i risultati
results = {}

# Addestramento e valutazione di ciascun modello
for model_name, model in models.items():
    print(f"\n--- {model_name} ---")
    
    # Addestramento del modello
    model.fit(X_train_smote, y_train_smote)
    
    # Predizione
    y_pred = model.predict(X_test_smote)
    
    # Report di classificazione
    acc = accuracy_score(y_test_smote, y_pred)
    print(f"Accuracy: {acc:.2f}")
    print(classification_report(y_test_smote, y_pred, target_names=surface_mapping.keys()))
    
    # Matrice di confusione
    conf_matrix = confusion_matrix(y_test_smote, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=surface_mapping.keys(), yticklabels=surface_mapping.keys())
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f"Confusion Matrix - {model_name}")
    plt.show()
    
    # Salva i risultati
    results[model_name] = {
        "accuracy": acc,
        "classification_report": classification_report(y_test_smote, y_pred, target_names=surface_mapping.keys(), output_dict=True),
        "conf_matrix": conf_matrix
    }

# Stampa dei risultati finali
print("\n--- Riepilogo Risultati ---")
for model_name, result in results.items():
    print(f"{model_name}: Accuracy = {result['accuracy']:.2f}")
    
#TODO: stampare curve validazione


In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import numpy as np

# Supponendo che y_test e y_pred_proba siano definiti:
# - y_test: target originali
# - y_pred_proba: probabilità predette per ciascuna classe

# Binarizzazione dei target per la gestione multiclasse
n_classes = len(np.unique(y_test_smote))
y_test_binarized = label_binarize(y_test_smote, classes=np.arange(n_classes))

# Calcolo della curva ROC e AUC per ogni classe
fpr = {}
tpr = {}
roc_auc = {}

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_binarized[:, i], model.predict_proba(X_test_smote)[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Calcolo delle curve micro-average e macro-average
# Micro-average: considera tutti i punti come un unico problema binario
fpr["micro"], tpr["micro"], _ = roc_curve(y_test_binarized.ravel(), model.predict_proba(X_test_smote).ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Macro-average: media delle AUC di tutte le classi
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
mean_tpr = np.zeros_like(all_fpr)

for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot delle curve ROC
plt.figure(figsize=(10, 8))

# Curve per ogni classe
colors = ['blue', 'green', 'red']
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2, label=f'Classe {i} (AUC = {roc_auc[i]:.2f})')

# Curve macro e micro average
plt.plot(fpr["micro"], tpr["micro"],
         label=f'Micro-average (AUC = {roc_auc["micro"]:.2f})', color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label=f'Macro-average (AUC = {roc_auc["macro"]:.2f})', color='navy', linestyle=':', linewidth=4)

# Linea casuale
plt.plot([0, 1], [0, 1], color='grey', linestyle='--', lw=2)

# Etichette e legenda
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC Multiclasse')
plt.legend(loc='lower right')
plt.grid()
plt.show()
